In [2]:
import numpy as np

In [3]:
def lambda_t(t):
    # Definimos los rangos de acuerdo con la tabla proporcionada
    tabla = [
        (0, 1, 1.979),
        (1, 2, 1.972),
        (2, 3, 4.054),
        (3, 4, 4.088),
        (4, 5, 5.998),
        (5, 6, 7.970),
        (6, 7, 10.015),
        (7, 8, 13.922),
        (8, 9, 15.979),
        (9, 10, 12.109),
        (10, 11, 11.913),
        (11, 12, 15.789),
        (12, 13, 19.744),
        (13, 14, 24.030),
        (14, 15, 29.776),
        (15, 16, 26.026),
        (16, 17, 22.102),
        (17, 18, 18.033),
        (18, 19, 17.814),
        (19, 20, 12.224),
        (20, 21, 9.957),
        (21, 22, 5.730),
        (22, 23, 6.152),
        (23, 24, 1.943)
    ]
    
    # Iteramos sobre la tabla para encontrar el rango correspondiente
    for valor_min, valor_max, lambda_valor in tabla:
        if valor_min <= t < valor_max:
            return lambda_valor
    
    # Si el valor no está en ningún rango, devolver None o un mensaje de error
    return None

def obtener_mayor_tasa(lambda_t):
    # Suponemos que t varía entre 0 y 24, basado en los valores de la tabla
    max_tasa = max(lambda_t(t) for t in range(24))
    return max_tasa

def generar_poisson_no_homogeneo(lambda_t, T_max):
    t = 0
    lambda_u = obtener_mayor_tasa(lambda_t)
    eventos = list()
    while True:
        # Paso 1: Generar u1 ~ U(0,1)
        u1 = np.random.uniform(0, 1)
        
        # Paso 2: Actualizar t
        t -= (1 / lambda_u) * np.log(u1)
        
        # Paso 3: Generar u2 ~ U(0,1)
        u2 = np.random.uniform(0, 1)
        
        # Paso 4: Comparar u2 con λ(t)/λ_u
        if u2 <= lambda_t(t%24) / lambda_u:
            eventos.append(t)
            if t > T_max:
                return np.array(eventos)

def generar_servicios(llegadas, tasa=0.1663156965801959):
    return np.random.exponential(tasa, len(llegadas))

def generar_incertidumbre(lambda_t, T_max):
    llegadas = generar_poisson_no_homogeneo(lambda_t, T_max)
    return list(llegadas), list(generar_servicios(llegadas))

In [4]:
class Cliente:
    CONTADOR_ID = 1  # Variable de clase para contar el número de clientes creados
    INFINITO = np.inf  # Constante para tiempos que no se han definido aún

    def __init__(self, tiempo_sistema, tiempo_atencion):
        self.id = Cliente.CONTADOR_ID  # Asigna un ID único al cliente
        self.tiempo_arribo = tiempo_sistema  # Tiempo en el que el cliente llega al sistema
        self.tiempo_atencion = tiempo_atencion  # Tiempo requerido para atender al cliente
        self.tiempo_inicio_servicio = Cliente.INFINITO  # Tiempo en el que inicia el servicio
        self.tiempo_fin_servicio = Cliente.INFINITO  # Tiempo en el que termina el servicio
        self.tiempo_espera_cola = 0  # Inicialmente, el tiempo de espera en cola es 0
        self.tiempo_total_sistema = Cliente.INFINITO  # Tiempo total que el cliente pasa en el sistema
        Cliente.CONTADOR_ID += 1  # Incrementa el contador de clientes

    def registrar_inicio_atencion(self, tiempo_sistema):
        """
        Registra el inicio de la atención y calcula el tiempo de espera en la cola.
        """
        if self.tiempo_inicio_servicio != Cliente.INFINITO:
            raise ValueError("El inicio de atención ya ha sido registrado.")
        
        self.tiempo_inicio_servicio = tiempo_sistema
        self.tiempo_espera_cola = self.tiempo_inicio_servicio - self.tiempo_arribo
        self.tiempo_fin_servicio = tiempo_sistema + self.tiempo_atencion

    def finalizar_atencion(self):
        """
        Registra el final de la atención, calcula el tiempo de salida del sistema y el tiempo total en el sistema.
        """
        if self.tiempo_inicio_servicio == Cliente.INFINITO:
            raise ValueError("El inicio de atención no ha sido registrado.")
        
        self.tiempo_total_sistema = self.tiempo_fin_servicio - self.tiempo_arribo

    def __str__(self):
        return (f"Cliente {self.id}: Arribo={self.tiempo_arribo}, "
                f"Inicio de Atención={self.tiempo_inicio_servicio}, "
                f"Fin de Atención={self.tiempo_fin_servicio}, "
                f"Espera en Cola={self.tiempo_espera_cola}, "
                f"Tiempo Total en Sistema={self.tiempo_total_sistema}")


In [5]:
import numpy as np

class SistemaCajas:

    def __init__(self, n_cajas):
        self.n_cajas = n_cajas  # Número de cajas disponibles
        self.clientes = []  # Lista de clientes en el sistema
        self.proximo_cliente = None  # Cliente que será atendido a continuación
        self.proximo_servicio = np.inf  # Tiempo del próximo servicio

    def agregar_cliente(self, tiempo_sistema, cliente):
        """
        Agrega un cliente al sistema y registra el inicio de su atención.
        Si el cliente tiene un tiempo de fin de servicio más corto que el actual,
        actualiza el próximo cliente y el próximo tiempo de servicio.
        """
        if not self.chequeo_capacidad():
            raise ValueError("No hay capacidad disponible en las cajas para un nuevo cliente.")
        
        self.clientes.append(cliente)
        cliente.registrar_inicio_atencion(tiempo_sistema)

        # Actualizar el próximo cliente a ser atendido si es el más rápido en terminar
        if self.proximo_cliente is None or cliente.tiempo_fin_servicio < self.proximo_servicio:
            self.proximo_cliente = cliente
            self.proximo_servicio = cliente.tiempo_fin_servicio

    def chequeo_capacidad(self):
        """Verifica si hay capacidad en las cajas para un nuevo cliente."""
        return len(self.clientes) < self.n_cajas

    def finalizar_atencion(self):
        """
        Finaliza la atención del cliente con el tiempo de fin de servicio más cercano,
        actualiza el sistema y retorna el cliente que ha finalizado su atención.
        """
        if self.proximo_cliente is None:
            raise ValueError("No hay clientes para finalizar la atención.")

        egreso = self.proximo_cliente
        egreso.finalizar_atencion()
        self.clientes.remove(egreso)

        # Resetear el próximo cliente y servicio
        self.proximo_cliente = None
        self.proximo_servicio = np.inf

        # Buscar el próximo cliente a ser atendido
        for cliente in self.clientes:
            if cliente.tiempo_fin_servicio < self.proximo_servicio:
                self.proximo_cliente = cliente
                self.proximo_servicio = cliente.tiempo_fin_servicio

        return egreso

In [6]:
class ListaEspera:

    def __init__(self, capacidad):
        """
        Inicializa la lista de espera con una capacidad máxima.
        """
        self.capacidad = capacidad
        self.clientes = []
    
    def chequeo_capacidad(self):
        """
        Verifica si hay espacio disponible en la lista de espera.
        """
        return len(self.clientes) < self.capacidad
    
    def agregar_cliente(self, cliente):
        """
        Agrega un cliente a la lista de espera si hay capacidad.
        """
        if not self.chequeo_capacidad():
            raise ValueError("No hay capacidad en la lista de espera para agregar más clientes.")
        
        self.clientes.append(cliente)
    
    def eliminar_cliente(self):
        """
        Elimina y devuelve el primer cliente en la lista de espera.
        Si la lista está vacía, lanza una excepción.
        """
        if not self.clientes:
            raise IndexError("No hay clientes en la lista de espera para eliminar.")
        
        return self.clientes.pop(0) 

In [7]:
class ListaAtendidos:

    def __init__(self):
        """
        Inicializa una lista para almacenar clientes que han sido atendidos.
        """
        self.clientes = []
    
    def agregar_cliente(self, cliente):
        """
        Agrega un cliente a la lista de clientes atendidos.
        """
        self.clientes.append(cliente)
    
    def contar_clientes(self):
        """
        Retorna la cantidad de clientes que han sido atendidos.
        """
        return len(self.clientes)
    
    def obtener_clientes(self):
        """
        Retorna la lista de todos los clientes que han sido atendidos.
        """
        return self.clientes
    
    def __str__(self):
        """
        Retorna una representación en cadena de la lista de clientes atendidos.
        """
        return f"Lista de clientes atendidos: {self.clientes}"

In [8]:
class Simulacion:

    def __init__(self, seed, T_max, n_cajas, cap_lista, generar_incertidumbre):
        """
        Inicializa la simulación con los parámetros dados.

        Parámetros:
        seed: Semilla para la generación de números aleatorios.
        T_max: Tiempo máximo de simulación.
        n_cajas: Número de cajas disponibles en el sistema.
        cap_lista: Capacidad de la lista de espera.
        generar_llegadas: Función para generar tiempos de llegada de los clientes.
        generar_servicios: Función para generar tiempos de servicio de los clientes.
        """
        np.random.seed(seed)  # Establece la semilla para la generación aleatoria
        self.T_max = T_max  # Tiempo máximo de simulación
        self.llegadas = []  # Lista para almacenar los tiempos entre llegadas
        self.servicios = []  # Lista para almacenar los tiempos de servicio
        self.sistema_cajas = SistemaCajas(n_cajas)  # Inicializa el sistema de cajas
        self.lista_espera = ListaEspera(cap_lista)  # Inicializa la lista de espera
        self.lista_atendidos = ListaAtendidos()  # Inicializa la lista de clientes atendidos
        self.clientes = []  # Lista para almacenar los clientes que llegan al sistema
        self.rechazos = 0  # Contador de rechazos cuando la lista de espera está llena

        # Listas para rastrear los tiempos y eventos
        self.t_inicio = []
        self.t_fin = []
        self.id_cliente = []
        self.nodo = []

        # Genera los tiempos de llegada y servicio hasta alcanzar el tiempo máximo de simulación
        self.llegadas, self.servicios = generar_incertidumbre(lambda_t, self.T_max)
    
    def simular(self):
        """
        Ejecuta la simulación del sistema de colas.
        """
        self.T = 0  # Inicializa el tiempo del sistema
        A = self.llegadas.pop(0) if self.llegadas else np.inf  # Tiempo de la próxima llegada
        S = np.inf  # Tiempo del próximo servicio finalizado, inicializado en infinito

        # Bucle principal de simulación
        while self.T <= self.T_max:

            E = min(A, S)  # El próximo evento es el más temprano entre una llegada o un servicio finalizado

            if E == A:  # Si el próximo evento es una llegada
                self.T = A  # Actualiza el tiempo del sistema al tiempo de llegada
                if self.llegadas:  # Si quedan más llegadas
                    A = self.llegadas.pop(0)  # Programa la siguiente llegada
                else:
                    A = np.inf  # Si no hay más llegadas, establece A en infinito

                # Crea un nuevo cliente con el tiempo actual y su tiempo de servicio
                nuevo_cliente = Cliente(self.T, self.servicios.pop(0))
                self.clientes.append(nuevo_cliente)  # Almacena el cliente en la lista

                if self.sistema_cajas.chequeo_capacidad():  # Si hay espacio en las cajas
                    self.sistema_cajas.agregar_cliente(self.T, nuevo_cliente)  # Atiende al cliente
                    S = self.sistema_cajas.proximo_servicio  # Actualiza el tiempo del próximo servicio finalizado

                elif self.lista_espera.chequeo_capacidad():  # Si no hay espacio en las cajas pero sí en la lista de espera
                    self.lista_espera.agregar_cliente(nuevo_cliente)  # Agrega el cliente a la lista de espera

                    # Registra el evento de adición a la lista de espera
                    self.t_inicio.append(self.T)
                    self.t_fin.append(self.T)
                    self.id_cliente.append(nuevo_cliente.id)
                    self.nodo.append('Lista de Espera')

                else:  # Si no hay espacio ni en las cajas ni en la lista de espera
                    self.rechazos += 1  # Incrementa el contador de rechazos

                    # Registra el evento de rechazo
                    self.t_inicio.append(self.T)
                    self.t_fin.append(self.T)
                    self.id_cliente.append(nuevo_cliente.id)
                    self.nodo.append('Rechazo')

            else:  # Si el próximo evento es un servicio finalizado
                self.T = S  # Actualiza el tiempo del sistema al tiempo de fin de servicio
                egreso = self.sistema_cajas.finalizar_atencion()  # Finaliza la atención y obtiene el cliente egresado
                self.lista_atendidos.agregar_cliente(egreso)  # Agrega el cliente a la lista de atendidos

                # Registra el evento de fin de servicio
                self.t_inicio.append(egreso.tiempo_inicio_servicio)
                self.t_fin.append(egreso.tiempo_fin_servicio)
                self.id_cliente.append(egreso.id)
                self.nodo.append('Atención en Cajas')

                if self.lista_espera.clientes:  # Si hay clientes en la lista de espera
                    atender = self.lista_espera.eliminar_cliente()  # Elimina el cliente de la lista de espera
                    self.sistema_cajas.agregar_cliente(self.T, atender)  # Atiende al cliente desde la lista de espera
                    S = self.sistema_cajas.proximo_servicio  # Actualiza el tiempo del próximo servicio finalizado

                else:
                    S = np.inf  # Si no hay más clientes en servicio, establece S en infinito

In [9]:
simulacion = Simulacion(1, 24*1100, 3, 5, generar_incertidumbre)

In [10]:
simulacion.simular()

In [11]:
import pandas as pd

In [12]:
# Crear el DataFrame
df = pd.DataFrame({
    'ID Cliente': simulacion.id_cliente,
    'Nodo': simulacion.nodo,
    'Tiempo Inicio': simulacion.t_inicio,
    'Tiempo Fin': simulacion.t_fin
})

In [13]:
# Function to convert hours to dd:hh:mm:ss
def convert_hours_to_ddhhmmss(hours):
    # Convert to timedelta
    td = pd.to_timedelta(hours, unit='h')
    
    # Extract days, hours, minutes, and seconds
    days = td.dt.days
    hrs = td.dt.components.hours
    minutes = td.dt.components.minutes
    secs = td.dt.components.seconds
    
    # Format into dd:hh:mm:ss
    return days.astype(str).str.zfill(2) + ':' + \
           hrs.astype(str).str.zfill(2) + ':' + \
           minutes.astype(str).str.zfill(2) + ':' + \
           secs.astype(str).str.zfill(2)

In [14]:
df['Tiempo Inicio'] = convert_hours_to_ddhhmmss(df['Tiempo Inicio'])
df['Tiempo Fin'] = convert_hours_to_ddhhmmss(df['Tiempo Fin'])

In [15]:
df[df['ID Cliente'] == 111]

,ID Cliente,Nodo,Tiempo Inicio,Tiempo Fin
172,111,Rechazo,00:12:00:44,00:12:00:44


In [16]:
df

,ID Cliente,Nodo,Tiempo Inicio,Tiempo Fin
0,1,Atención en Cajas,00:00:55:06,00:01:09:14
1,2,Atención en Cajas,00:01:35:22,00:01:36:26
2,3,Atención en Cajas,00:02:44:36,00:02:52:41
3,6,Atención en Cajas,00:03:09:59,00:03:12:02
4,4,Atención en Cajas,00:02:52:30,00:03:12:37
...,...,...,...,...
490577,328602,Atención en Cajas,1099:21:23:29,1099:21:25:33
490578,328603,Atención en Cajas,1099:21:27:05,1099:21:36:08
490579,328604,Atención en Cajas,1099:21:55:22,1099:22:04:19
490580,328605,Atención en Cajas,1099:22:17:02,1099:22:17:10


In [17]:
# Convertir las columnas 'Inicio' y 'Fin' a formato de tiempo compatible
def convert_to_timedelta(time_str):
    d, h, m, s = map(int, time_str.split(':'))
    return pd.Timedelta(days=d, hours=h, minutes=m, seconds=s)

In [18]:
df['Inicio'] = df['Tiempo Inicio'].apply(convert_to_timedelta)
df['Fin'] = df['Tiempo Fin'].apply(convert_to_timedelta)

In [19]:
# Filtrar filas donde 'Inicio' es mayor que 95 días
filtered_df = df[df['Inicio'] > pd.Timedelta(days=95)]

In [20]:
# Filtrar clientes con Nodo 'Rechazo'
clientes_rechazo = filtered_df[filtered_df['Nodo'] == 'Rechazo']

# Calcular el porcentaje
porcentaje_rechazo = (len(clientes_rechazo) / (filtered_df['ID Cliente'].max()-filtered_df['ID Cliente'].min())) * 100

print(f"Porcentaje de clientes con Nodo 'Rechazo': {porcentaje_rechazo:.4f}%")

Porcentaje de clientes con Nodo 'Rechazo': 14.2668%


In [21]:
# Filtrar clientes con Nodo 'Lista de Espera'
clientes_lista_espera = filtered_df[filtered_df['Nodo'] == 'Lista de Espera']

# Calcular el porcentaje
porcentaje_lista_espera = (len(clientes_lista_espera) / (filtered_df['ID Cliente'].max()-filtered_df['ID Cliente'].min())) * 100

print(f"Porcentaje de clientes con Nodo 'Lista de Espera': {porcentaje_lista_espera:.4f}%")

Porcentaje de clientes con Nodo 'Lista de Espera': 49.2390%


In [22]:
# Filtrar filas con 'Lista de Espera' y 'Atención en Cajas'
lista_espera = filtered_df[filtered_df['Nodo'] == 'Lista de Espera'].set_index('ID Cliente')
atencion_cajas = filtered_df[filtered_df['Nodo'] == 'Atención en Cajas'].set_index('ID Cliente')

# Unir ambos DataFrames por 'ID Cliente'
merged = lista_espera[['Fin']].join(atencion_cajas[['Inicio']], lsuffix='_ListaEspera', rsuffix='_AtencionCajas')

# Calcular la diferencia
merged['Diferencia'] = merged['Inicio'] - merged['Fin']
merged['Diferencia'].mean()

Timedelta('0 days 00:10:13.313195806')